In [1]:
import torch
import os

In [2]:
frgens = os.listdir("./custom_output/data/mtn1_fr-en_bfs_recom_2_-1_False_0.4_True_False_4_5_rcb_0.9_0.0_0.9/")

In [3]:
len(frgens)

85317

In [50]:
# sample - I am his friend was
# we want the following mask 
"""
1 0 0 0 0 - I 
1 1 0 0 0 - am 
1 1 1 0 1 - his
1 1 1 1 1 - friend
1 0 0 0 1 - was
"""
# start from adjacency (previous nodes)
"""
0 0 0 0 0 - I
1 0 0 0 0 - am
0 1 0 0 0 - his
0 0 1 0 0 - friend
1 0 0 0 0 - was
"""
back_adjac = torch.tensor([
    [0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 1],
    [0, 0, 1, 0, 0],
    [1, 0, 0, 0, 0],
])

In [48]:
# get adjacency from flat canvas of tokens, with previous tokens
def adj_mat(flat_canv, mlen):
    # TODO write this function
    mask = torch.zeros((mlen, mlen))
    # have a cutoff after the calculated limit to circumvent extra computation
    for row in range(mlen):
        for p in flat_canv[row].prevs:
            if p.canvpos<mlen:
                mask[row][p.canvpos] = 1
    return mask

# ensure that token canvas has the right values for mask creation
def mask_prep_canv(canv, lim):
    maxpos = -1
    # we're taking in DLReverseNodes
    ind = 0
    for c in canv:
        # maybe should be an assert instead to ward off weirdness?
        c.canvpos = ind
        maxpos = max(maxpos, c.pos)
        # only include as a test, remove later
        for prev in c.prevs:
            assert prev in canv
    return maxpos  

# get connectivity matrix from flat canvas of tokens
# by exponentiating reverse connected adjacency
def connect_mat(flat_canv):
    # TODO something to get length, while considering space from inp tokens
    mlen = 5
    #back_adjac = adj_mat(flat_canv)
    tot = back_adjac
    tmp = back_adjac
    # TODO we need length of longest path in graph, make sure canvpos is updated
    # can do in one O(N) pass I think
    #maxlen = 3
    while torch.sum((tot[:, 0]>0))<(mlen-1):
        tmp = torch.mm(back_adjac, tmp)
        tot += tmp
    tot = tot+ torch.eye(mlen)
    return (tot>0).int()

In [41]:
torch.sum((back_adjac[:, 0]>0))

tensor(2)

In [36]:
back_adjac

tensor([[0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 1, 0, 0, 1],
        [0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0]])

In [52]:
back_adjac = torch.tensor([
    [0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 1],
    [0, 0, 1, 0, 0],
    [1, 0, 0, 0, 0],
])
connect_mat(None)

tensor([[1, 0, 0, 0, 0],
        [1, 1, 0, 0, 0],
        [1, 1, 1, 0, 1],
        [1, 1, 1, 1, 1],
        [1, 0, 0, 0, 1]], dtype=torch.int32)

In [ ]:
import torch
# TODO go into github to get the older version of this file

# TODO more sanity checking on whether or not we're recording the right number of branches     
def get_adjac_mat(pgraph):
    res = []
    for node in pgraph:
        tmp = []
        for p in pgraph:
            if str(p['id']) in node['nexts']:
                tmp.append(1)
            else:
                tmp.append(0)
        res.append(torch.tensor(tmp))
    return torch.stack(res)

def get_poslist(processed):
    res = []
    for p in processed:
        res.append(p['pos'])
    return res

def conmat(adj, longestpath):
    res = torch.zeros_like(adj)
    adj = torch.triu(adj)
    start = adj
    for i in range(0, longestpath):
        if i>0:
            start = torch.mm(start, adj)
        res = res + start
    return res

# fixes input to now account for [SEP] and [CLS] I believe
def correct_mask_sep(mat):
    for i in range(len(mat)):
        mat[i][i] = 1
    # TODO need to remove for stuff
    # fixed = torch.nn.functional.pad(input=mat, pad=(1, 1, 1, 1), mode='constant', value=1)
    return mat

def connect_mat(pgraph):
    conm = conmat(get_adjac_mat(pgraph), max(get_poslist(pgraph)))
    conm = conm + torch.transpose(conm, 0, 1)

    cmat =  (conm>0).float()
    # handle things to be able to take when really big input
    respad = None
    MAX =512
    # we're within 498
    if len(cmat) < MAX-1:
        # get the version with CLS and SEP
        cmat = correct_mask_sep(cmat)
        # copy that onto fixed input: the rest is padding
        respad = torch.zeros((MAX, MAX))
        cpmax = len(cmat)
        for i in range(cpmax):
            for j in range(cpmax):
                respad[i][j] = cmat[i][j]
    else:
        # TODO made changes here
        # we're over the limit, first get truncated version
        respad = torch.zeros((MAX, MAX))
        # make truncated matrix
        for i in range(MAX):
            for j in range(MAX):
                respad[i][j] = cmat[i][j]
        # do CLS / SEP tokens on that 
        respad = correct_mask_sep(respad)
        
    #respad[respad==0] = -float('inf')
    return respad #ones_padding(respad)